# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np

# Uploading Data

In [2]:
files = ['AIRPORTS',
'AREA',
'BUDGET',
'continent',
'CURACCT',
'DEBTFOREIGNERS',
'EDUCATIONSPEND',
'electricconsupmtion',
'emissions',
'EXPORTS',
'fossilelectricity',
'GDP',
'GDPGROWTH',
'GDPPERCAP',
'hydroelectricity',
'IMPORTS',
'INDUSTPRODGR',
'internetusers',
'LIFEXP',
'MARMERCHANT',
'MIGRATION',
'MILIEXPENDITURE',
'naturalgasconsumption',
'nuclearelectricity',
'otherrenewableelectricity',
'petroleumconsumption',
'POPGROWTH',
'POPULATION',
'PUBLICDEBT',
'RAILWAYS',
'ROADWAYS',
'UNEMPLOYMENT',
'WATERWAYS',
'WORKFORCE']

In [3]:
pd_dict = {}
for file in files:
    pd_dict[file] = pd.read_csv('{}.csv'.format(file))

In [4]:
pd_dict['AIRPORTS']

,Unnamed: 0,COUNTRY,AIRPORTS,YEAR
0,1,United States,"13,513",2013
1,2,Brazil,"4,093",2013
2,3,Mexico,"1,714",2013
3,4,Canada,"1,467",2013
4,5,Russia,"1,218",2013
...,...,...,...,...
233,234,Saint Barthelemy,1,2013
234,235,Saint Martin,1,2013
235,236,Sint Maarten,1,2013
236,237,Tuvalu,1,2013


In [5]:
pd_dict['continent']

,continent_name,country_name
0,Asia,"Afghanistan, Islamic Republic of"
1,Europe,"Albania, Republic of"
2,Antarctica,Antarctica (the territory South of 60 deg S)
3,Africa,"Algeria, People's Democratic Republic of"
4,Oceania,American Samoa
...,...,...
257,Africa,"Zambia, Republic of"
258,Oceania,Disputed Territory
259,Asia,Iraq-Saudi Arabia Neutral Zone
260,Asia,United Nations Neutral Zone


In [6]:
base_df = pd_dict['continent']
area_df = pd_dict['AREA']
petrol_df = pd_dict['petroleumconsumption']
users_df = pd_dict['internetusers']
hydro_df = pd_dict['hydroelectricity']
nuclear_df = pd_dict['nuclearelectricity']
otherrenew_df = pd_dict['otherrenewableelectricity']
mili_df = pd_dict['MILIEXPENDITURE']

In [7]:
base_df.rename(columns = {'country_name':'Country'}, inplace = True)
base_df.merge(area_df, on = 'Country', how = 'left')

,continent_name,Country,Index,Area_sqKm
0,Asia,"Afghanistan, Islamic Republic of",NaN,NaN
1,Europe,"Albania, Republic of",NaN,NaN
2,Antarctica,Antarctica (the territory South of 60 deg S),NaN,NaN
3,Africa,"Algeria, People's Democratic Republic of",NaN,NaN
4,Oceania,American Samoa,216.0,224
...,...,...,...,...
257,Africa,"Zambia, Republic of",NaN,NaN
258,Oceania,Disputed Territory,NaN,NaN
259,Asia,Iraq-Saudi Arabia Neutral Zone,NaN,NaN
260,Asia,United Nations Neutral Zone,NaN,NaN


In [14]:
base_df.head()

,continent_name,Country
0,Asia,"Afghanistan, Islamic Republic of"
1,Europe,"Albania, Republic of"
2,Antarctica,Antarctica (the territory South of 60 deg S)
3,Africa,"Algeria, People's Democratic Republic of"
4,Oceania,American Samoa


In [15]:
mili_df.head()

,Unnamed: 0,COUNTRY,% OF GDP,YEAR,Unnamed: 4
0,1,Saudi Arabia,8.78,2018,NaN
1,2,Oman,8.17,2018,NaN
2,3,United Arab Emirates,5.70,2016,NaN
3,4,Algeria,5.27,2018,NaN
4,5,Kuwait,5.06,2018,NaN


In [18]:
final_df = base_df
final_df['Area_sqKm'] = final_df['Area_sqKm'].replace(',', '')
final_df['refined_petroleum_consumption_barrels_per_day'] = final_df['refined_petroleum_consumption_barrels_per_day'].replace(',', '')
final_df['number_of_internet_users'] = final_df['number_of_internet_users'].replace(',', '')
final_df['hydro_percentage_of_electricity_production'] = final_df['hydro_percentage_of_electricity_production'].replace(',', '')
final_df['Area_sqKm'] = final_df['Area_sqKm'].astype(float)
final_df['refined_petroleum_consumption_barrels_per_day'] = final_df['refined_petroleum_consumption_barrels_per_day'].astype(float)
final_df['number_of_internet_users'] = final_df['number_of_internet_users'].astype(float)
final_df['hydro_percentage_of_electricity_production'] = final_df['hydro_percentage_of_electricity_production'].astype(float)

KeyError: 'Area_sqKm'